<a href="https://colab.research.google.com/github/abrarahmed56/Ansible-Assessment/blob/main/Ansible_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
from googleapiclient.discovery import build

# Build the Drive service
drive_service = build('drive', 'v3')

# Define the metadata for the new Google Docs document
file_metadata = {
    'name': 'Abrar Ahmed Ansible Assessment- Meeting Notes using Colab',
    'mimeType': 'application/vnd.google-apps.document'
}

file = drive_service.files().create(body=file_metadata, fields='id,webViewLink').execute()

print(f"Google Docs document created with ID: {file.get('id')}")
print(f"Access it at: {file.get('webViewLink')}")

Google Docs document created with ID: 1Oc9KjsoAXzIMGzmj5djH664FzE_iTFQaM6C14Ovo760
Access it at: https://docs.google.com/document/d/1Oc9KjsoAXzIMGzmj5djH664FzE_iTFQaM6C14Ovo760/edit?usp=drivesdk


In [3]:
notes = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

In [7]:
formatted_lines = []

formatting_markers = [
    ("# ", "HEADING_1"),
    ("## ", "HEADING_2"),
    ("### ", "HEADING_3"),
    ("* ", "BULLET"),
    ("- [ ] ", "CHECKBOX"),
    ("- ", "DASH"),
]
# Any character between "@" and the below will be bolded
# For example, in "@Abrar:", bolding will end at the second "r"
tag_enders = [" ", ":", "&"]

bullet_spacing = []
bullet_sections = []
footer_started = False
last_paragraph_style = None

for line in notes.split("\n"):
  found_format = False
  for marker, style in formatting_markers:
    if line.strip().startswith(marker):
      stripped_line_text = line[line.find(marker)+len(marker):].strip()
      if style == "BULLET":
        current_bullet_spacing = line.find(marker)
        while bullet_spacing and bullet_spacing[-1] >= current_bullet_spacing:
          bullet_spacing.pop()
        bullet_tabs = "\t" * len(bullet_spacing)
        bullet_spacing.append(current_bullet_spacing)
        formatted_lines.append({
            "text": bullet_tabs + stripped_line_text + "\n",
            "paragraph_style": style
          })
      else:
        if bullet_spacing:
          bullet_spacing = []
        last_paragraph_style = style
        formatted_lines.append({
            "text": stripped_line_text + "\n",
            "paragraph_style": style
          })
      found_format = True
      break
  if not found_format:
    formatted_line = {"text": line.strip() + "\n"}
    if footer_started:
      formatted_line['text_style'] = 'ITALIC'
    if line.startswith("---"):
      footer_started = True
    formatted_lines.append(formatted_line)
    last_paragraph_style = None

# Remove newlines before and after headings
i = 0
while i < len(formatted_lines):
  if 'paragraph_style' in formatted_lines[i] and formatted_lines[i]['paragraph_style'].startswith('HEADING'):
    if i > 1 and formatted_lines[i-1] == {'text': '\n'}:
      formatted_lines.pop(i-1)
      i -= 1
    if i < len(formatted_lines) - 1 and formatted_lines[i+1] == {'text': '\n'}:
      formatted_lines.pop(i+1)
  i += 1

formatted_lines

[{'text': 'Product Team Sync - May 15, 2023\n',
  'paragraph_style': 'HEADING_1'},
 {'text': 'Attendees\n', 'paragraph_style': 'HEADING_2'},
 {'text': 'Sarah Chen (Product Lead)\n', 'paragraph_style': 'DASH'},
 {'text': 'Mike Johnson (Engineering)\n', 'paragraph_style': 'DASH'},
 {'text': 'Anna Smith (Design)\n', 'paragraph_style': 'DASH'},
 {'text': 'David Park (QA)\n', 'paragraph_style': 'DASH'},
 {'text': 'Agenda\n', 'paragraph_style': 'HEADING_2'},
 {'text': '1. Sprint Review\n', 'paragraph_style': 'HEADING_3'},
 {'text': 'Completed Features\n', 'paragraph_style': 'BULLET'},
 {'text': '\tUser authentication flow\n', 'paragraph_style': 'BULLET'},
 {'text': '\tDashboard redesign\n', 'paragraph_style': 'BULLET'},
 {'text': '\tPerformance optimization\n', 'paragraph_style': 'BULLET'},
 {'text': '\t\tReduced load time by 40%\n', 'paragraph_style': 'BULLET'},
 {'text': '\t\tImplemented caching solution\n', 'paragraph_style': 'BULLET'},
 {'text': 'Pending Items\n', 'paragraph_style': 'BUL

In [8]:
requests = []
cursor = 1
processing_bullets = False
bullet_style = 'BULLET_DISC_CIRCLE_SQUARE' # default bullet style
bullets_beginning = -1
tabs_to_replace = 0

for line in formatted_lines:
  requests.append({
      'insertText': {
          'location': {'index': cursor},
          'text': line['text'],
        },
    })
  if 'paragraph_style' in line:
    if line['paragraph_style'] == 'BULLET' or line['paragraph_style'] == 'DASH':
      if not processing_bullets:
        processing_bullets = True
        bullets_beginning = cursor
        if line['paragraph_style'] == 'BULLET':
          bullet_style = 'BULLET_DISC_CIRCLE_SQUARE'
        else:
          bullet_style = 'BULLET_ARROW_DIAMOND_DISC'
      tabs_to_replace += line['text'].count("\t")
    elif line['paragraph_style'] == 'CHECKBOX':
      requests.append({
          'createParagraphBullets': {
              'range': {
                  'startIndex': cursor,
                  'endIndex': cursor + len(line['text'])
              },
              'bulletPreset': 'BULLET_CHECKBOX',
          }
      })
    else:
      if processing_bullets:
        requests.append({
            'createParagraphBullets': {
                'range': {'startIndex': bullets_beginning, 'endIndex': cursor},
                'bulletPreset': bullet_style,
            },
        })
        cursor -= tabs_to_replace
        tabs_to_replace = 0
        processing_bullets = False
        bullet_style = 'BULLET_DISC_CIRCLE_SQUARE'
        bullets_beginning = -1
      requests.append({
          'updateParagraphStyle': {
              'paragraphStyle': { 'namedStyleType': line['paragraph_style'] },
              'range': {
                  'startIndex': cursor,
                  'endIndex': cursor + len(line['text']) - 1,
              },
              'fields': '*',
            },
      })
  else:
    if processing_bullets:
      requests.append({
          'createParagraphBullets': {
              'range': {'startIndex': bullets_beginning, 'endIndex': cursor},
              'bulletPreset': bullet_style,
          },
      })
      cursor -= tabs_to_replace
      tabs_to_replace = 0
      processing_bullets = False
      bullet_style = 'BULLET_DISC_CIRCLE_SQUARE'
      bullets_beginning = -1
    if 'text_style' in line:
      requests.append({
          'updateTextStyle': {
              'range': {
                  'startIndex': cursor,
                  'endIndex': cursor + len(line['text']),
              },
              'textStyle': {'italic': line['text_style'] == 'ITALIC'},
              'fields': '*',
          },
      })
  if "@" in line['text']:
    indices_to_bold = []
    start_index = 0
    while "@" in line['text'][start_index:]:
      start_index = line['text'].find("@", start_index)
      end_index = start_index + 1
      while end_index < len(line['text']) and line['text'][end_index] not in tag_enders:
        end_index += 1
      if end_index == -1:
        end_index = len(line['text'])
      indices_to_bold.append((start_index, end_index))
      start_index = end_index
    for start, end in indices_to_bold:
      requests.append({
          'updateTextStyle': {
              'range': {'startIndex': cursor + start, 'endIndex': cursor + end},
              'textStyle': {'bold': True},
              'fields': '*'
          }
      })
  cursor += len(line['text'])

requests

[{'insertText': {'location': {'index': 1},
   'text': 'Product Team Sync - May 15, 2023\n'}},
 {'updateParagraphStyle': {'paragraphStyle': {'namedStyleType': 'HEADING_1'},
   'range': {'startIndex': 1, 'endIndex': 33},
   'fields': '*'}},
 {'insertText': {'location': {'index': 34}, 'text': 'Attendees\n'}},
 {'updateParagraphStyle': {'paragraphStyle': {'namedStyleType': 'HEADING_2'},
   'range': {'startIndex': 34, 'endIndex': 43},
   'fields': '*'}},
 {'insertText': {'location': {'index': 44},
   'text': 'Sarah Chen (Product Lead)\n'}},
 {'insertText': {'location': {'index': 70},
   'text': 'Mike Johnson (Engineering)\n'}},
 {'insertText': {'location': {'index': 97}, 'text': 'Anna Smith (Design)\n'}},
 {'insertText': {'location': {'index': 117}, 'text': 'David Park (QA)\n'}},
 {'insertText': {'location': {'index': 133}, 'text': 'Agenda\n'}},
 {'createParagraphBullets': {'range': {'startIndex': 44, 'endIndex': 133},
   'bulletPreset': 'BULLET_ARROW_DIAMOND_DISC'}},
 {'updateParagraphStyl

In [11]:
docs_service = build('docs', 'v1')
try:
  result = docs_service.documents().batchUpdate(
    documentId=file.get('id'),
    body={'requests': requests}
  ).execute()
  print("Batch update successful!")
except Exception as e:
  print("Batch update failed, make sure notes were parsed correctly and "
        "requests are well-formed")
  print(e)

Batch update successful!
